# Basic Text Classification with Naive Bayes
***
In the mini-project, you'll learn the basics of text analysis using a subset of movie reviews from the rotten tomatoes database. You'll also use a fundamental technique in Bayesian inference, called Naive Bayes. This mini-project is based on [Lab 10 of Harvard's CS109](https://github.com/cs109/2015lab10) class.  Please free to go to the original lab for additional exercises and solutions.

In [1]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from six.moves import range

# Setup Pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("poster")

# Table of Contents

* [Rotten Tomatoes Dataset](#Rotten-Tomatoes-Dataset)
    * [Explore](#Explore)
* [The Vector Space Model and a Search Engine](#The-Vector-Space-Model-and-a-Search-Engine)
    * [In Code](#In-Code)
* [Naive Bayes](#Naive-Bayes)
    * [Multinomial Naive Bayes and Other Likelihood Functions](#Multinomial-Naive-Bayes-and-Other-Likelihood-Functions)
    * [Picking Hyperparameters for Naive Bayes and Text Maintenance](#Picking-Hyperparameters-for-Naive-Bayes-and-Text-Maintenance)
* [Interpretation](#Interpretation)


## Rotten Tomatoes Dataset

In [3]:
critics = pd.read_csv('./critics.csv')
#let's drop rows with missing quotes
critics = critics[~critics.quote.isnull()]
critics.head()

critic  fresh    imdb     publication                                              quote review_date  rtid      title
1         Derek Adams  fresh  114709        Time Out  So ingenious in concept, design and execution ...  2009-10-04  9559  Toy story
2     Richard Corliss  fresh  114709   TIME Magazine                  The year's most inventive comedy.  2008-08-31  9559  Toy story
3         David Ansen  fresh  114709        Newsweek  A winning animated feature that has something ...  2008-08-18  9559  Toy story
4       Leonard Klady  fresh  114709         Variety  The film sports a provocative and appealing st...  2008-06-09  9559  Toy story
5  Jonathan Rosenbaum  fresh  114709  Chicago Reader  An entertaining computer-generated, hyperreali...  2008-03-10  9559  Toy story

### Explore

In [4]:
n_reviews = len(critics)
n_movies = critics.rtid.unique().size
n_critics = critics.critic.unique().size


print("Number of reviews: {:d}".format(n_reviews))
print("Number of critics: {:d}".format(n_critics))
print("Number of movies:  {:d}".format(n_movies))

Number of reviews: 15561
Number of critics: 623
Number of movies:  1921


In [5]:
df = critics.copy()
df['fresh'] = df.fresh == 'fresh'
grp = df.groupby('critic')
counts = grp.critic.count()  # number of reviews by each critic
means = grp.fresh.mean()     # average freshness for each critic

means[counts > 100].hist(bins=10, edgecolor='w', lw=1)
plt.xlabel("Average Rating per critic")
plt.ylabel("Number of Critics")
plt.yticks([0, 2, 4, 6, 8, 10]);

<div class="span5 alert alert-info">
<h3>Exercise Set I</h3>
<br/>
<b>Exercise:</b> Look at the histogram above. Tell a story about the average ratings per critic. What shape does the distribution look like? What is interesting about the distribution? What might explain these interesting things?
</div>

From the Rotten Tomatoes dataset histogram, we can see that the most prolific critics on the site with more than 100 reviews to their name tend to review 60% of the movies they review with a 'fresh' rating. The most critical critic of the bunch is an individual who only rates 40% of the movies they watch with 'fresh' while the most lenient critics vote nearly 80% of their movies with a certified fresh rating. The double peaked bell curve from this histogram implies that there's a camp of people who provide mostly lukewarm reception to the movies they review. For the most part, however, movie critics are fair in their reviewing process and score the movies they review with a 'fresh' rating.

## The Vector Space Model and a Search Engine

All the diagrams here are snipped from [*Introduction to Information Retrieval* by Manning et. al.]( http://nlp.stanford.edu/IR-book/) which is a great resource on text processing. For additional information on text mining and natural language processing, see [*Foundations of Statistical Natural Language Processing* by Manning and Schutze](http://nlp.stanford.edu/fsnlp/).

Also check out Python packages [`nltk`](http://www.nltk.org/), [`spaCy`](https://spacy.io/), [`pattern`](http://www.clips.ua.ac.be/pattern), and their associated resources. Also see [`word2vec`](https://en.wikipedia.org/wiki/Word2vec).

Let us define the vector derived from document $d$ by $\bar V(d)$. What does this mean? Each document is treated as a vector containing information about the words contained in it. Each vector has the same length and each entry "slot" in the vector contains some kind of data about the words that appear in the document such as presence/absence (1/0), count (an integer) or some other statistic. Each vector has the same length because each document shared the same vocabulary across the full collection of documents -- this collection is called a *corpus*.

To define the vocabulary, we take a union of all words we have seen in all documents. We then just associate an array index with them. So "hello" may be at index 5 and "world" at index 99.

Suppose we have the following corpus:

`A Fox one day spied a beautiful bunch of ripe grapes hanging from a vine trained along the branches of a tree. The grapes seemed ready to burst with juice, and the Fox's mouth watered as he gazed longingly at them.`

Suppose we treat each sentence as a document $d$. The vocabulary (often called the *lexicon*) is the following:

$V = \left\{\right.$ `a, along, and, as, at, beautiful, branches, bunch, burst, day, fox, fox's, from, gazed, grapes, hanging, he, juice, longingly, mouth, of, one, ready, ripe, seemed, spied, the, them, to, trained, tree, vine, watered, with`$\left.\right\}$

Then the document

`A Fox one day spied a beautiful bunch of ripe grapes hanging from a vine trained along the branches of a tree`

may be represented as the following sparse vector of word counts:

$$\bar V(d) = \left( 4,1,0,0,0,1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,2,1,0,1,0,0,1,0,0,1,1,1,0,0 \right)$$

or more succinctly as

`[(0, 4), (1, 1), (5, 1), (6, 1), (7, 1), (9, 1), (10, 1), (12, 1), (14, 1), (15, 1), (20, 2), (21, 1), (23, 1),`
`(26, 1), (29,1), (30, 1), (31, 1)]`

along with a dictionary

``
{
    0: a, 1: along, 5: beautiful, 6: branches, 7: bunch, 9: day, 10: fox, 12: from, 14: grapes, 15: hanging, 19: mouth, 20: of, 21: one, 23: ripe, 24: seemed, 25: spied, 26: the, 29:trained, 30: tree, 31: vine, 
}
``

Then, a set of documents becomes, in the usual `sklearn` style, a sparse matrix with rows being sparse arrays representing documents and columns representing the features/words in the vocabulary.

Notice that this representation loses the relative ordering of the terms in the document. That is "cat ate rat" and "rat ate cat" are the same. Thus, this representation is also known as the Bag-Of-Words representation.

Here is another example, from the book quoted above, although the matrix is transposed here so that documents are columns:

![novel terms](terms.png)

Such a matrix is also catted a Term-Document Matrix. Here, the terms being indexed could be stemmed before indexing; for instance, `jealous` and `jealousy` after stemming are the same feature. One could also make use of other "Natural Language Processing" transformations in constructing the vocabulary. We could use Lemmatization, which reduces words to lemmas: work, working, worked would all reduce to work. We could remove "stopwords" from our vocabulary, such as common words like "the". We could look for particular parts of speech, such as adjectives. This is often done in Sentiment Analysis. And so on. It all depends on our application.

From the book:
>The standard way of quantifying the similarity between two documents $d_1$ and $d_2$  is to compute the cosine similarity of their vector representations $\bar V(d_1)$ and $\bar V(d_2)$:

$$S_{12} = \frac{\bar V(d_1) \cdot \bar V(d_2)}{|\bar V(d_1)| \times |\bar V(d_2)|}$$

![Vector Space Model](vsm.png)


>There is a far more compelling reason to represent documents as vectors: we can also view a query as a vector. Consider the query q = jealous gossip. This query turns into the unit vector $\bar V(q)$ = (0, 0.707, 0.707) on the three coordinates below. 

![novel terms](terms2.png)

>The key idea now: to assign to each document d a score equal to the dot product:

$$\bar V(q) \cdot \bar V(d)$$

Then we can use this simple Vector Model as a Search engine.

### In Code

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

text = ['Hop on pop', 'Hop off pop', 'Hop Hop hop']
print("Original text is\n{}".format('\n'.join(text)))

vectorizer = CountVectorizer(min_df=0)

# call `fit` to build the vocabulary
vectorizer.fit(text)

# call `transform` to convert text to a bag of words
x = vectorizer.transform(text)

# CountVectorizer uses a sparse array to save memory, but it's easier in this assignment to 
# convert back to a "normal" numpy array
x = x.toarray()

print("")
print("Transformed text vector is \n{}".format(x))

# `get_feature_names` tracks which word is associated with each column of the transformed x
print("")
print("Words for each feature:")
print(vectorizer.get_feature_names())

# Notice that the bag of words treatment doesn't preserve information about the *order* of words, 
# just their frequency

Original text is
Hop on pop
Hop off pop
Hop Hop hop

Transformed text vector is 
[[1 0 1 1]
 [1 1 0 1]
 [3 0 0 0]]

Words for each feature:
['hop', 'off', 'on', 'pop']


In [7]:
def make_xy(critics, vectorizer=None):
    if vectorizer is None:
        vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(critics.quote)
    X = X.tocsc()  # some versions of sklearn return COO format
    y = (critics.fresh == 'fresh').values.astype(np.int)
    return X, y
X, y = make_xy(critics)

## Naive Bayes

From Bayes' Theorem, we have that

$$P(c \vert f) = \frac{P(c \cap f)}{P(f)}$$

where $c$ represents a *class* or category, and $f$ represents a feature vector, such as $\bar V(d)$ as above. **We are computing the probability that a document (or whatever we are classifying) belongs to category *c* given the features in the document.** $P(f)$ is really just a normalization constant, so the literature usually writes Bayes' Theorem in context of Naive Bayes as

$$P(c \vert f) \propto P(f \vert c) P(c) $$

$P(c)$ is called the *prior* and is simply the probability of seeing class $c$. But what is $P(f \vert c)$? This is the probability that we see feature set $f$ given that this document is actually in class $c$. This is called the *likelihood* and comes from the data. One of the major assumptions of the Naive Bayes model is that the features are *conditionally independent* given the class. While the presence of a particular discriminative word may uniquely identify the document as being part of class $c$ and thus violate general feature independence, conditional independence means that the presence of that term is independent of all the other words that appear *within that class*. This is a very important distinction. Recall that if two events are independent, then:

$$P(A \cap B) = P(A) \cdot P(B)$$

Thus, conditional independence implies

$$P(f \vert c)  = \prod_i P(f_i | c) $$

where $f_i$ is an individual feature (a word in this example).

To make a classification, we then choose the class $c$ such that $P(c \vert f)$ is maximal.

There is a small caveat when computing these probabilities. For [floating point underflow](http://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html) we change the product into a sum by going into log space. This is called the LogSumExp trick. So:

$$\log P(f \vert c)  = \sum_i \log P(f_i \vert c) $$

There is another caveat. What if we see a term that didn't exist in the training data? This means that $P(f_i \vert c) = 0$ for that term, and thus $P(f \vert c)  = \prod_i P(f_i | c) = 0$, which doesn't help us at all. Instead of using zeros, we add a small negligible value called $\alpha$ to each count. This is called Laplace Smoothing.

$$P(f_i \vert c) = \frac{N_{ic}+\alpha}{N_c + \alpha N_i}$$

where $N_{ic}$ is the number of times feature $i$ was seen in class $c$, $N_c$ is the number of times class $c$ was seen and $N_i$ is the number of times feature $i$ was seen globally. $\alpha$ is sometimes called a regularization parameter.

### Multinomial Naive Bayes and Other Likelihood Functions

Since we are modeling word counts, we are using variation of Naive Bayes called Multinomial Naive Bayes. This is because the likelihood function actually takes the form of the multinomial distribution.

$$P(f \vert c) = \frac{\left( \sum_i f_i \right)!}{\prod_i f_i!} \prod_{f_i} P(f_i \vert c)^{f_i} \propto \prod_{i} P(f_i \vert c)$$

where the nasty term out front is absorbed as a normalization constant such that probabilities sum to 1.

There are many other variations of Naive Bayes, all which depend on what type of value $f_i$ takes. If $f_i$ is continuous, we may be able to use *Gaussian Naive Bayes*. First compute the mean and variance for each class $c$. Then the likelihood, $P(f \vert c)$ is given as follows

$$P(f_i = v \vert c) = \frac{1}{\sqrt{2\pi \sigma^2_c}} e^{- \frac{\left( v - \mu_c \right)^2}{2 \sigma^2_c}}$$

<div class="span5 alert alert-info">
<h3>Exercise Set II</h3>

<p><b>Exercise:</b> Implement a simple Naive Bayes classifier:</p>

<ol>
<li> split the data set into a training and test set
<li> Use `scikit-learn`'s `MultinomialNB()` classifier with default parameters.
<li> train the classifier over the training set and test on the test set
<li> print the accuracy scores for both the training and the test sets
</ol>

What do you notice? Is this a good classifier? If not, why not?
</div>

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Train-test split on the data
X, y = make_xy(critics)
X = X.toarray()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit on the training data and predict on the test data
mnb = MultinomialNB()
mnb.fit(Xtrain, ytrain)
y_pred = mnb.predict(Xtest)

# Measure the accuracy of MNB on training data and testing data
train_accuracy = mnb.score(Xtrain, ytrain)
test_accuracy = mnb.score(Xtest, ytest)

# Print the Results
print(f'Training Accuracy: {train_accuracy: .4f}')
print(f'Testing Accuracy: {test_accuracy: .4f}')

Training Accuracy:  0.9202
Testing Accuracy:  0.7889


This classifier is decent. The classifier can identify the 'rotten' / 'fresh' classifiers with 79% accuracy by only looking at the contents of each review in tandem with their classifier. The model might see some improvement through hyperparameter selection on the alpha value.

### Picking Hyperparameters for Naive Bayes and Text Maintenance

We need to know what value to use for $\alpha$, and we also need to know which words to include in the vocabulary. As mentioned earlier, some words are obvious stopwords. Other words appear so infrequently that they serve as noise, and other words in addition to stopwords appear so frequently that they may also serve as noise.

First, let's find an appropriate value for `min_df` for the `CountVectorizer`. `min_df` can be either an integer or a float/decimal. If it is an integer, `min_df` represents the minimum number of documents a word must appear in for it to be included in the vocabulary. If it is a float, it represents the minimum *percentage* of documents a word must appear in to be included in the vocabulary. From the documentation:

>min_df: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

<div class="span5 alert alert-info">
<h3>Exercise Set III</h3>

<p><b>Exercise:</b> Construct the cumulative distribution of document frequencies (df). The $x$-axis is a document count $x_i$ and the $y$-axis is the percentage of words that appear less than $x_i$ times. For example, at $x=5$, plot a point representing the percentage or number of words that appear in 5 or fewer documents.</p>

<p><b>Exercise:</b> Look for the point at which the curve begins climbing steeply. This may be a good value for `min_df`. If we were interested in also picking `max_df`, we would likely pick the value where the curve starts to plateau. What value did you choose?</p>
</div>

In [9]:
# Convert the word frequency array X to a DataFrame
X_df = pd.DataFrame(X)

# Compute the sums of all the columns of X, sort them, and extract their values
X_values = X_df.sum(axis=0).sort_values().values

# Perform a log transform on X since the range of values goes from 1 to 16805 occurrences
X_cdf = np.log(X_values)
y_cdf = np.arange(1, len(X_cdf)+1)/len(X_cdf)

# Plot the ECDF
_ = plt.plot(X_cdf, y_cdf, marker = '.', linestyle='none')
_ = plt.xlabel('Log Word Frequency')
_ = plt.ylabel('ECDF')
_ = plt.title('ECDF for Word Frequency in Rotten Tomatoes Reviews')
_ = plt.margins(0.02)
plt.show()


# Print lowest and highest word frequencies
print(f'Lowest word frequency: {min(X_values)}')
print(f'Highest word frequency: {max(X_values)}')


Lowest word frequency: 1
Highest word frequency: 16805


In [10]:
# Number of unique words
num_unique = len(X_values)

# Number of single word occurrences
single_occurrences = len(X_values[X_values == 1])

# Percentage of single words to total words
prop_single = single_occurrences / num_unique

# Index where 50% and 99% of the unique word occurrences exists
index_50 = int(0.50 * len(X_values))
index_99 = int(0.99 * len(X_values))

# Print results
print(f'Number of Unique Words: {num_unique}')
print(f'Number of Single Occurrences: {single_occurrences}')
print(f'Proportion of Single Occurrences: {prop_single : .2f}')
print(f'50th Percentile of Word Occurences: {X_values[index_50]}')
print(f'99th Percentile of Word Occurences: {X_values[index_99]}')

Number of Unique Words: 22417
Number of Single Occurrences: 9552
Proportion of Single Occurrences:  0.43
50th Percentile of Word Occurences: 2
99th Percentile of Word Occurences: 145


In order to return a legible ECDF, a log transform is applied to the word frequency data. We do this because the word frequencies range from a minimum of 1 and a maximum of 16805 occurrences. 99% of the words are shared among 145 documents, so without a log transform, the higher range of word frequencies would stretch the x-axis to the point where 99% of the data is lumped together to accomodate for the 1%.

So, looking at our transformed ECDF, we can see that there is a major spike at a word occurence of 1. Analyzing this point further, we can see that 43% of the words seen in the dataset only appear once. Since these words are unique to their documents, they don't add any information and should be removed in our classifier definition. Furthermore, 99% of the words seen in this dataset only occur 145 times. Words occurring too often can also be a source of noise since appearing so often means that the word is shared among many documents and doesn't add any helpful information to our classifier either. As a result, we should set ```min_df = 2``` and ```max_df = 145```.

---

The parameter $\alpha$ is chosen to be a small value that simply avoids having zeros in the probability computations. This value can sometimes be chosen arbitrarily with domain expertise, but we will use K-fold cross validation. In K-fold cross-validation, we divide the data into $K$ non-overlapping parts. We train on $K-1$ of the folds and test on the remaining fold. We then iterate, so that each fold serves as the test fold exactly once. The function `cv_score` performs the K-fold cross-validation algorithm for us, but we need to pass a function that measures the performance of the algorithm on each fold. 

In [11]:
from sklearn.model_selection import KFold
def cv_score(clf, X, y, scorefunc):
    result = 0.
    nfold = 5
    for train, test in KFold(nfold).split(X): # split data into train/test groups, 5 times
        clf.fit(X[train], y[train]) # fit the classifier, passed is as clf.
        result += scorefunc(clf, X[test], y[test]) # evaluate score function on held-out data
    return result / nfold # average

We use the log-likelihood as the score here in `scorefunc`. The higher the log-likelihood, the better. Indeed, what we do in `cv_score` above is to implement the cross-validation part of `GridSearchCV`.

The custom scoring function `scorefunc` allows us to use different metrics depending on the decision risk we care about (precision, accuracy, profit etc.) directly on the validation set. You will often find people using `roc_auc`, precision, recall, or `F1-score` as the scoring function.

In [12]:
def log_likelihood(clf, x, y):
    prob = clf.predict_log_proba(x)
    rotten = y == 0
    fresh = ~rotten
    return prob[rotten, 0].sum() + prob[fresh, 1].sum()

We'll cross-validate over the regularization parameter $\alpha$.

<div class="span5 alert alert-info">
<h3>Exercise Set IV</h3>

<p><b>Exercise:</b> What does using the function `log_likelihood` as the score mean? What are we trying to optimize for?</p>

<p><b>Exercise:</b> Without writing any code, what do you think would happen if you choose a value of $\alpha$ that is too high?</p>

<p><b>Exercise:</b> Using the skeleton code below, find the best values of the parameter `alpha`, and use the value of `min_df` you chose in the previous exercise set. Use the `cv_score` function above with the `log_likelihood` function for scoring.</p>
</div>

The log-likelihood score is a value from zero to one that sums up the likelihood that each given word correctly identifies the classifier. Log-likelihood takes a more holistic approach to an accuracy score for this example because it takes into account every word in the lexicon and its ability to predict the binary 'rotten' / 'fresh' classification.

If the alpha value is too high for a Multinomial Naive Bayes classifier, then the weight for the number of times the feature occurs in class c and the number of times class c appears in the dataset is undermined by the size of alpha. $$P(f_i \vert c) = \frac{N_{ic}+\alpha}{N_c + \alpha N_i}$$

Alpha should be sufficiently small so that the value of alpha doesn't diminish the effects of class c but sufficiently large so that a new word introduced to the lexicon is still taken into consideration when determining class c.

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# the grid of parameters to search over
alphas = [.1, 1, 5, 10, 50]
best_min_df = 2

# Find the best value for alpha and the best classifier
best_alpha = None
maxscore = -np.inf

# Make the X / y vectors and train/test splits using the CountVectorizer
vectorizer = CountVectorizer(min_df=best_min_df)       
X, y = make_xy(critics, vectorizer)
X = X.toarray()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

for alpha in alphas:
    
    # Instantiate the classifier with the alpha value
    mnb = MultinomialNB(alpha=alpha)
    
    # Cross-validate and find the score using log_likelihood function
    score = cv_score(mnb, Xtrain, ytrain, log_likelihood)
    
    # Update the best_alpha value if the score outperformed the previous alpha
    if score > maxscore:
        maxscore = score
        best_alpha = alpha

In [14]:
print("alpha: {}".format(best_alpha))

alpha: 1


<div class="span5 alert alert-info">
<h3>Exercise Set V: Working with the Best Parameters</h3>

<p><b>Exercise:</b> Using the best value of  `alpha` you just found, calculate the accuracy on the training and test sets. Is this classifier better? Why (not)?</p>

</div>

In [15]:
clf = MultinomialNB(alpha=best_alpha).fit(Xtrain, ytrain)

#your turn. Print the accuracy on the test and training dataset
training_accuracy = clf.score(Xtrain, ytrain)
test_accuracy = clf.score(Xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

Accuracy on training data: 0.901751
Accuracy on test data:     0.789913


The alpha value for ```best_alpha = 1``` is the same as the default alpha value for Multinomial Naive Bayes in scikit-learn. As a result, the classifier on the testing data yields the same accuracy on the testing data as our originial classifier defined above.

In [16]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest, clf.predict(Xtest)))

[[ 850  359]
 [ 295 1609]]


## Interpretation

### What are the strongly predictive features?

We use a neat trick to identify strongly predictive features (i.e. words). 

* first, create a data set such that each row has exactly one feature. This is represented by the identity matrix.
* use the trained classifier to make predictions on this matrix
* sort the rows by predicted probabilities, and pick the top and bottom $K$ rows

In [17]:
# Collect the feature names from the vectorizer transformation
words = np.array(vectorizer.get_feature_names())

# Create an identity matrix where each row represents a single word
x = np.eye(Xtest.shape[1])

# Predict the log-likelihood of each word
probs = clf.predict_log_proba(x)[:, 0]

# Sort the indices by the values in the probability vector
ind = np.argsort(probs)

# Find the words with the 10 highest and lowest probability scores
good_words = words[ind[:10]]
bad_words = words[ind[-10:]]

# Find the probabilities with the 10 highest and lowest scores
good_prob = probs[ind[:10]]
bad_prob = probs[ind[-10:]]

print("Good words\t     P(fresh | word)")
for w, p in zip(good_words, good_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))
    
print("Bad words\t     P(fresh | word)")
for w, p in zip(bad_words, bad_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

Good words	     P(fresh | word)
             delight 0.97
          remarkable 0.97
           absorbing 0.97
           energetic 0.96
            intimate 0.96
            captures 0.96
              wilder 0.96
                myth 0.95
         outstanding 0.95
              richly 0.95
Bad words	     P(fresh | word)
       unfortunately 0.08
            sluggish 0.07
        unsatisfying 0.07
             witless 0.06
               bland 0.06
            tiresome 0.06
          uninspired 0.06
             unfunny 0.05
           pointless 0.04
                lame 0.04


<div class="span5 alert alert-info">
<h3>Exercise Set VI</h3>

<p><b>Exercise:</b> Why does this method work? What does the probability for each row in the identity matrix represent</p>

</div>

This method starts by extracting the words from the vectorizer and creating an identity matrix with dimensionality equal to the words in the lexicon. From there, ```clf.predict_log_proba(x)``` finds the log-likelihoods of all the words given the classifier fit to the ```Xtrain``` and ```ytrain``` datasets. The output of ```clf.predict_log_proba(x)``` is a 128321 * 2 array where the first vector corresponds to the class 'fresh' and the second vector corresponds to the class 'rotten'. Each row of this array is the log-likelihood scores for each individual word. Since we are interested in 'fresh' classification, we use the first vector of the array. Sorting the indices by the highest probability of 'fresh' classification using the ```np.argsort(probs)``` function can then be used to find the indices for the top and bottom ten 'fresh' words. The result is then printed to display.

The above exercise is an example of *feature selection*. There are many other feature selection methods. A list of feature selection methods available in `sklearn` is [here](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection). The most common feature selection technique for text mining is the chi-squared $\left( \chi^2 \right)$ [method](http://nlp.stanford.edu/IR-book/html/htmledition/feature-selectionchi2-feature-selection-1.html).

### Prediction Errors

We can see mis-predictions as well.

In [18]:
x, y = make_xy(critics, vectorizer)

prob = clf.predict_proba(x)[:, 0]
predict = clf.predict(x)

bad_rotten = np.argsort(prob[y == 0])[:5]
bad_fresh = np.argsort(prob[y == 1])[-5:]

print("Mis-predicted Rotten quotes")
print('---------------------------')
for row in bad_rotten:
    print(critics[y == 0].quote.iloc[row])
    print("")

print("Mis-predicted Fresh quotes")
print('--------------------------')
for row in bad_fresh:
    print(critics[y == 1].quote.iloc[row])
    print("")

Mis-predicted Rotten quotes
---------------------------
The spirited cast of Jamie Lee Curtis, Kevin Kline and Monty Pythonites John Cleese and Michael Palin is willing. But this tossed-together trifle about an unassuming British zoo taken over by a money-mad Murdochian media mogul is weak.

This pacifist spirit of brotherhood echoes the heroics in Princess Mononoke and other anime titles, but the artistic gap between the Miyazaki masterpiece and this project is huge.

Anne Frank Remembered tells the audience very little about Anne, but the film speaks volumes concerning the problematic aspects of Holocaust representation.

Benefits from a lively lead performance by the miscast Denzel Washington but doesn't come within light years of the book, one of the greatest American autobiographies.

While Leone's vision still has a magnificent sweep, the film finally subsides to an emotional core that is sombre, even elegiac, and which centres on a man who is bent and broken by time, and finally

<div class="span5 alert alert-info">
<h3>Exercise Set VII: Predicting the Freshness for a New Review</h3>
<br/>
<div>
<b>Exercise:</b>
<ul>
<li> Using your best trained classifier, predict the freshness of the following sentence: *'This movie is not remarkable, touching, or superb in any way'*
<li> Is the result what you'd expect? Why (not)?
</ul>
</div>
</div>

In [19]:
sentence = ['This movie is not remarkable, touching, or superb in any way.']
sentence_transform = vectorizer.transform(sentence)

pred_sentence = clf.predict(sentence_transform)
pred_prob = clf.predict_proba(sentence_transform)[0][1]
print(f'Predicted Fresh Rating : {pred_prob*100:.2f}%')

Predicted Fresh Rating : 99.95%


We expected this sentence to be classified as a rotten rating, but instead, the classifier yielded a 99.95% probability fresh rating. We can attribute this misclassification to the sentiment of the sentence. Using ```CountVectorizer```, the Multinomial Naive Bayes classifier only looks at word frequencies of documents rated fresh / rotten for its classification. The word 'not' in the sentence negates all positive sentiment from the words 'remarkable', 'touching', and 'superb' which throws off the predictive model. Adjusting the ```CountVectorizer``` to account for negation of sentiment would allow our classifier to accurately predict the classification.

### Aside: TF-IDF Weighting for Term Importance

TF-IDF stands for 

`Term-Frequency X Inverse Document Frequency`.

In the standard `CountVectorizer` model above, we used just the term frequency in a document of words in our vocabulary. In TF-IDF, we weight this term frequency by the inverse of its popularity in all documents. For example, if the word "movie" showed up in all the documents, it would not have much predictive value. It could actually be considered a stopword. By weighing its counts by 1 divided by its overall frequency, we downweight it. We can then use this TF-IDF weighted features as inputs to any classifier. **TF-IDF is essentially a measure of term importance, and of how discriminative a word is in a corpus.** There are a variety of nuances involved in computing TF-IDF, mainly involving where to add the smoothing term to avoid division by 0, or log of 0 errors. The formula for TF-IDF in `scikit-learn` differs from that of most textbooks: 

$$\mbox{TF-IDF}(t, d) = \mbox{TF}(t, d)\times \mbox{IDF}(t) = n_{td} \log{\left( \frac{\vert D \vert}{\vert d : t \in d \vert} + 1 \right)}$$

where $n_{td}$ is the number of times term $t$ occurs in document $d$, $\vert D \vert$ is the number of documents, and $\vert d : t \in d \vert$ is the number of documents that contain $t$

In [20]:
# http://scikit-learn.org/dev/modules/feature_extraction.html#text-feature-extraction
# http://scikit-learn.org/dev/modules/classes.html#text-feature-extraction-ref
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(min_df=1, stop_words='english')
Xtfidf=tfidfvectorizer.fit_transform(critics.quote)

<div class="span5 alert alert-info">
<h3>Exercise Set VIII: Enrichment <b>(Optional)</b></h3>

<p>
There are several additional things we could try. Try some of these as exercises:
<ol>
<li> Build a Naive Bayes model where the features are n-grams instead of words. N-grams are phrases containing n words next to each other: a bigram contains 2 words, a trigram contains 3 words, and 6-gram contains 6 words. This is useful because "not good" and "so good" mean very different things. On the other hand, as n increases, the model does not scale well since the feature set becomes more sparse.
<li> Try a model besides Naive Bayes, one that would allow for interactions between words -- for example, a Random Forest classifier.
<li> Try adding supplemental features -- information about genre, director, cast, etc.
<li> Use word2vec or [Latent Dirichlet Allocation](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) to group words into topics and use those topics for prediction.
<li> Use TF-IDF weighting instead of word counts.
</ol>
</p>

<b>Exercise:</b> Try at least one of these ideas to improve the model (or any other ideas of your own). Implement here and report on the result. 
</div>